# Final Notebook Pt 3.1: Transit Overlays

**By:** Andrew Williams and Ben Brassette
    
**Description:** This notebook is building on earlier work for each neighborhood. Purpose is to intergrate bus lines and rail stations and our other remaining datasets. We are tackling points 1, 3, 6, and 7 in this notebook. Additonally, this notebook is building off of ealier transit notebooks which will serves as the foundation. 

**Expected Outputs**
* Overlay of rail stops and bus lines

**Areas Where We Need More Work**
* Coordinate maps and charts
* Create story map
* Clean Notebooks, with outputs leading to inssight for our paper

**Notes for self to advance project:**
1.  Biggest step forward, intergrating our notebooks in the remaining few weeks in a targeted fashion
2.  Overlay heavy rail and light rail stops
3.  Provide more accurate description of the areas through a spatial join
4. Compare with race/ethnicity and income data; we should create some side by side comparisons of transit use and race/eth and income
5. Query "high" transit tracts, cross reference with income and race/eth, potential to add spatial dimension of LA neighborhoods
6. Availability of cars in households
7. Bring a shaprer focus to Central LA and the San Fernando Valley

# Importing Libraries and Datasets

Importing libraries for gpd, plotting, and basepamps.

In [ ]:
import urllib.request, json 
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns 
import contextily as ctx

Import was successful.

Importing datasets: rail, bus, and bus frequency.

In [ ]:
rail_stops = gpd.read_file('Data/RailStations0715/Stations_All_0715.shp')

bus_freq = gpd.read_file('Data/means2work_freq/Existing_Bus_System.shp')

Shape files are ready for exploration now. 

# Bus Frequency 

This is new bus data. I imagine it will mirror much of what I had before from just the LA Metro bus lines, with the addition of frequency. I'm going to explore it a little but and see what I have to work with.

In [ ]:
type(bus_freq)

In [ ]:
bus_freq.info()

Lots of columns! I can vaugely picture what I this will say in terms of frequency.

In [ ]:
bus_freq.shape

In [ ]:
bus_freq.head(3)

In [ ]:
bus_freq.tail(3)

In terms of the the columns, I believe I'm seeing the number of times a bus is being in use during a particular period.

I would like to see what routes/bus-systems/tiers are available in this dataset.

In [ ]:
bus_freq['tier'].value_counts()

The biggest differnce I see between my previous Metro bus data was that the local lines were subdivided into two categories: CBD and Non-CBD. This one large category seems unruly, but if need be, I can query and sub divide it further. 

Next step is to create to sub-categories which will be easier to call than having an extroadinarly long syntaxt (though I should practice those as well)>

## Local Bus Freq

Will define and quickly map data to see results.

Pulling subset of data- local routes

In [ ]:
local_f = bus_freq[bus_freq.tier == 'Local']
local_f.sample(3)

Pulling subset of data- Rapid routes

In [ ]:
local_f = bus_freq[bus_freq.tier == 'Local']
local_f.sample(3)

This is great! Much better than my previous bus maps. I'm going to cycle through and finish defining these subcategoires. Next steps would include a query into which routes are the heavist used and then overlay that with transit census tracts. 

## Rapid Freq

Pulling subset of data- Rapid routes

In [ ]:
rapid_f = bus_freq[bus_freq.tier == 'Rapid']
rapid_f.sample(3)

Also revealing. I'm curious what the line on the far east of the map is and what its connections are to job centers, other lines. 

## Limited Frew

Pulling subset of data- Limited routes

In [ ]:
limited_f = bus_freq[bus_freq.tier == 'Limited']
limited_f.sample(3)

## Shuttle Freq

Pulling subset of data-- for Shuttles

In [ ]:
shuttle_f = bus_freq[bus_freq.tier == 'Shuttles']
shuttle_f.sample(3)

## Exp Freq

Pulling subset of data- Exp routes

In [ ]:
exp_f = bus_freq[bus_freq.tier == 'Express']
exp_f.sample(3)

## Busway Freq

Pulling subset of data- Busway routes

In [ ]:
bw_f = bus_freq[bus_freq.tier == 'Busway']
bw_f.sample(3)

# Rail Line Stations 

Now for some rail lines. Going to do a little exploration. I've done this before during one of our class labs and on my own, but that over a month ago and I'm no longer farmiliar with this dataset.

In [ ]:
type(rail_stops)

In [ ]:
rail_stops.shape

In [ ]:
rail_stops.head(3)

How many rail stops are there per line. 

In [ ]:
rail_stops['LINE'].value_counts()

This helpful. Though from previous research, I know rail is not heavily used by LA resdients, or used often at all. LA is a car city first, then a bus city. Still, I want to explore the associations with the use of high transit.

There are more robust options to do here, especially with folium. 

# Neighborhood Dataset

Downloading master datase, running usual checks, and trimming it down some more.

In [ ]:
just_nbh = gpd.read_file('neighborhoods_trimmed.geojson') 

In [ ]:
type(just_nbh)

In [ ]:
just_nbh.sample(3)

In [ ]:
just_nbh.columns.to_list()

In [ ]:
keep= ['geoid',
 'name',
 'Neighborho',
 'Median Inc',
 'Total Work',
 'Car Total',
 'Drove alon',
 'Carpooled',
 'Public tra',
 'Bus',
 'Subway or',
 'Long-dista',
 'Light rail',
 'Worked fro',
 '%Car Total',
 '%Drove alo',
 '%Carpooled',
 '%Public tr',
 '%Bus',
 '%Subway or',
 '%Long-dist',
 '%Light rai',
 '%Worked fr',
 'Total Pop',
 'White',
 'Black',
 'Native',
 'Asian',
 'Pacific Is',
 'Hispanic o',
 '%White',
 '%Black',
 '%Native',
 '%Asian',
 '%Pacific I',
 '%Hispanic',
 'geometry']

In [ ]:
just_nbh=just_nbh[keep]
just_nbh

In [ ]:
just_nbh.columns= ['geoid',
 'name',
 'Neighborhood',
 'Median Income',
 'Total Workers Commuting',
 'Car Total',
 'Drove alone',
 'Carpooled',
 'Public transportation',
 'Bus',
 'Subway or elevated il',
 'Long-distance train or commuter rail',
 'Light rail',
 'Worked from home',
 '%Car Total',
 '%Drove alone',
 '%Carpooled',
 '%Public transportation',
 '%Bus',
 '%Subway or elevated il',
 '%Long-distance train or commuter rail',
 '%Light rail',
 '%Worked from home',
 'Total Pop Race',
 'White',
 'Black',
 'Native',
 'Asian',
 'Pacific Islander',
 'Hispanic or Latino',
 '%White',
 '%Black',
 '%Native',
 '%Asian',
 '%Pacific Islander',
 '%Hispanic or Latino',
  'geometry',]

In [ ]:
just_nbh.sample(3)

# Bus Maps

First map will be just for looking at general bus routes overlayed on neighborhoods.

In [ ]:
just_nbh = just_nbh.to_crs(epsg=3857)

In [ ]:
minx, miny, maxx, maxy = just_nbh.geometry.total_bounds
print(minx)
print(maxx)
print(miny)
print(maxy)

In [ ]:
fig,ax = plt.subplots(figsize=(20,20))

just_nbh.plot(ax=ax, column='Neighborhood', color='yellow', zorder=15,lw=0.5)

bus_freq.plot(ax=ax, alpha=0.75, zorder=20, lw=1, legend=True)


fig.patch.set_facecolor('black')
ax.axis('off')
#ctx.add_basemap(ax,source=ctx.providers.CartoDB.Positron)

* Playing around woth colors...
* Central LA...well is centrally located. Bus routes go through Downtown in high frequency. 
* Well connected on a broad level, but how serivce actually opertates remains another question
* Strong street grid in SF Valeey neighborhoods, could indicate easier connectivity and access to bus routes, more research needed

Bus Frequencies 

In [ ]:
fig,ax = plt.subplots(figsize=(20,20))

just_nbh.plot(ax=ax, column='Neighborhood', color='pink', zorder=15,lw=0.5)

bus_freq.plot(ax=ax, column='ampk_1', cmap='Blues', alpha=0.75, zorder=20, lw=1, legend=True, scheme='NaturalBreaks')

ax.axis('off')
#ctx.add_basemap(ax,source=ctx.providers.CartoDB.Positron)

* Map show bus frequencies.
* Not as frequent buses running in DT or PU, but there are a greater number of routes going through those areas.
* Higher frequencies on the outskirts, especially in east LA, and in the SF Valley

Buses running over 25 times during peak AM times

In [ ]:
fig,ax = plt.subplots(figsize=(20,20))

just_nbh.plot(ax=ax, column='Neighborhood', color='pink', zorder=15,lw=0.5)

bus_freq[bus_freq['ampk_1']>25].plot(ax=ax, alpha=0.75, zorder=20, lw=1, legend=True, scheme='NaturalBreaks')

ax.axis('off')
#ctx.add_basemap(ax,source=ctx.providers.CartoDB.Positron)

* No connectivity in MC
* Very few high freq buses runnin DT, none in PY, but may be compensated by other routes
* More high freq buses in NH and PC

# A Closer Look

## Central Los Angeles

In [ ]:
just_nbh.shape

In [ ]:
minx, miny, maxx, maxy = just_nbh.geometry.total_bounds
print(minx)
print(maxx)
print(miny)
print(maxy)

Bus Percentages

In [ ]:
fig,ax = plt.subplots(figsize=(20,20))

just_nbh[(just_nbh['Neighborhood']=='Downtown')|(just_nbh['Neighborhood']=='Pico-Union')|(just_nbh['Neighborhood']=='Mid-City')].plot(ax=ax, column='%Bus', zorder=15,lw=0.5, legend=True, scheme='NaturalBreaks',)

bus_freq.plot(ax=ax, column='ampk_1', color='gray', alpha=0.75, zorder=20, lw=.5, legend=True, scheme='NaturalBreaks')

ax.axis('off')
#ctx.add_basemap(ax,source=ctx.providers.CartoDB.Positron)

* Seems well connected, but need a closer look

In [ ]:
#alt ctx: ctx.add_basemap(ax, source=ctx.providers.Stamen.TonerLite)

In [ ]:
#ctx.add_basemap(ax,source=ctx.providers.CartoDB.Positron)
#ax.set_xlim(minx+.45,maxx-20)
#ax.set_ylim(miny+20,maxy-10)

In [ ]:
fig,ax = plt.subplots(figsize=(20,20))

just_nbh[(just_nbh['Neighborhood']=='Downtown')|(just_nbh['Neighborhood']=='Pico-Union')|(just_nbh['Neighborhood']=='Mid-City')].plot(ax=ax, column='%Public transportation', zorder=15,lw=0.5, legend=True, scheme='NaturalBreaks')

bus_freq.plot(ax=ax, column='ampk_1', color='gray', alpha=0.75, zorder=20, lw=1,)

ax.set_xlim(minx+.45,maxx-20)
ax.set_ylim(miny+20,maxy-10)
#ctx.add_basemap(ax,source=ctx.providers.CartoDB.Positron)

* tons of lines going through DT, and PU, some going through MC
* Difficult again to determine pattern though overlay
* Central LA is well connected, but where are people going?

## San Fernando Valley

In [ ]:
fig,ax = plt.subplots(figsize=(20,20))

just_nbh[(just_nbh['Neighborhood']=='North Hollywood')|(just_nbh['Neighborhood']=='Panorama City')].plot(ax=ax, column='%Public transportation', zorder=10, legend=True, scheme='NaturalBreaks')

bus_freq.plot(ax=ax, column='ampk_1', alpha=0.75, zorder=20, lw=1, legend=True, color='gray')

ax.set_xlim(minx+.45,maxx-20)
ax.set_ylim(miny+20,maxy-10)
#ctx.add_basemap(ax,source=ctx.providers.CartoDB.Positron)

* Zoomed in versino of SF Valley
* Not as well served as Central LA, but a good number of buses are runnign through NH and PACI
* Again, difficult it discern patterns with just an overlay

In [ ]:
fig,ax = plt.subplots(figsize=(20,20))

just_nbh[(just_nbh['Neighborhood']=='North Hollywood')|(just_nbh['Neighborhood']=='Panorama City')].plot(ax=ax, column='%Public transportation', zorder=10, legend=True, scheme='NaturalBreaks')

bus_freq.plot(ax=ax, column='ampk_1', alpha=0.75, zorder=20, lw=1, legend=True, color='gray')

#ctx.add_basemap(ax,source=ctx.providers.CartoDB.Positron)

Zoomed out version to show perspective, not much detail can be gathered here

In [ ]:
fig,ax = plt.subplots(figsize=(20,20))

just_nbh[(just_nbh['Neighborhood']=='North Hollywood')|(just_nbh['Neighborhood']=='Panorama City')].plot(ax=ax, column='%Bus', zorder=10, legend=True, scheme='NaturalBreaks')

bus_freq.plot(ax=ax, column='ampk_1', color='orange', alpha=0.75, zorder=20, lw=1,)

#ctx.add_basemap(ax,source=ctx.providers.CartoDB.Positron)
ax.set_xlim(minx+.45,maxx-20)
ax.set_ylim(miny+20,maxy-10)

Same map as above with different color choices. I like this, but the larger map looks better with gray and I would like to stay consistant.

# Rails

In [ ]:
rail_stops.shape

In [ ]:
rail_stops.head()

In [ ]:
rail_stops=rail_stops.to_crs(epsg=3857)

Let's plot some rail stops!

In [ ]:
fig, ax = plt.subplots(figsize=(20, 15))

rail_stops.plot(column='LINE', 
            alpha=0.75, 
           zorder=20,
            ax=ax,
            legend=True
           )

just_nbh.boundary.plot(ax=ax, color='black',zorder=10,lw=1)

#ctx.add_basemap(ax,source=ctx.providers.CartoDB.Positron)

For some reason, this took longer than it needed
* Subway as whole seems sparse, at least compared to the T in BOS
* Central LA seems very connected from just an overlay
* Mid-City, minues one tract, and Panorama City seems very unconnected

In [ ]:
fig, ax = plt.subplots(figsize=(20, 15))

rail_stops.plot(column='LINE', 
            alpha=0.75, 
            ax=ax,
                zorder=20,
            legend=True
           )

just_nbh.plot(ax=ax, column="Neighborhood", cmap='tab10', zorder=15 ,lw=0.5)


#ctx.add_basemap(ax,source=ctx.providers.CartoDB.Positron)

Same map as above, but different context

## Zoomed In

SF Valley

In [ ]:
fig, ax = plt.subplots(figsize=(20, 15))

rail_stops.plot(column='LINE', 
            color='red',
            alpha=0.75, 
            zorder=20,
            ax=ax,
            legend=True
           )

just_nbh[(just_nbh['Neighborhood']=='North Hollywood')|(just_nbh['Neighborhood']=='Panorama City')].plot(ax=ax, column='%Subway or elevated il', zorder=10, legend=True, scheme='NaturalBreaks')

ax.set_xlim(minx+.45,maxx-20)
ax.set_ylim(miny+20,maxy-10)
#ctx.add_basemap(ax,source=ctx.providers.CartoDB.Positron)

* Makes sense that at least one census tract uses more subway than the rest
* Dismal subway rates a whole
* Curious why there are two tracts north in North Hollywood using sub more. Still low rates nonetheless

Downtown

In [ ]:
fig, ax = plt.subplots(figsize=(20, 15))

rail_stops.plot(column='LINE', 
            color='red',
            alpha=0.75, 
            zorder=20,
            ax=ax,
            legend=True
           )

just_nbh[(just_nbh['Neighborhood']=='Downtown')|(just_nbh['Neighborhood']=='Pico-Union')|(just_nbh['Neighborhood']=='Mid-City')].plot(ax=ax, column='%Subway or elevated il', zorder=10, legend=True, scheme='NaturalBreaks')

ax.set_xlim(minx+.45,maxx-20)
ax.set_ylim(miny+20,maxy-10)
#ctx.add_basemap(ax,source=ctx.providers.CartoDB.Positron)

* Better connectivity and closer to subway stops, but subway is hardly used
* First inclication is that the subway doesn't takt this population, in Downtown at least, where it needs to go